In [1]:
import numpy as np
import pandas as pd
#_____________________________________________________________________________

from scripts.colors import bcolors as bc

In [2]:
file = './datas/Immoweb_data_ok_maite.csv'
data = pd.read_csv(file)
data.head()

,Id,Locality,House or appartment?,Subtype,Price,Type of sale,Number of rooms,Living area,Fully equipped kitchen,Furnished,Open fire,Terrace,Area of terrace,Garden,Area of garden,Land surface,Number of facades,Swimming pool,State of building
0,10233388,Uccle,HOUSE,HOUSE,1295000.0,BUY_REGULAR,4.0,290.0,1,None,0,1,15,None,None,230,2,0,GOOD
1,10311296,Woluwe-Saint-Pierre,APARTMENT,APARTMENT,150000.0,PUBLIC_SALE,2.0,65.0,0,None,0,1,None,None,None,None,None,None,None
2,10310451,Anderlecht,APARTMENT,APARTMENT,357500.0,BUY_REGULAR,3.0,119.0,0,None,0,1,16,None,None,None,4,None,NaN
3,10309870,Uccle,APARTMENT,APARTMENT,295000.0,BUY_REGULAR,2.0,140.0,0,None,0,None,None,None,None,None,2,None,TO_BE_DONE_UP
4,10309546,Uccle,APARTMENT,APARTMENT,199000.0,BUY_REGULAR,1.0,61.0,0,None,0,1,2,None,None,None,2,None,NaN


1. Check for duplicated rows in the df

In [ ]:
data.duplicated()

# data_duplicated = data[data.duplicated(subset='Id') == True]
# data_duplicated.head(15)


2. Remove data from apartment_group and house_group

In [28]:
data[data['House or appartment?'] == 'APARTMENT_GROUP'].count()
data.drop(data[data['House or appartment?'] == 'APARTMENT_GROUP'].index, inplace=True)
data[data['House or appartment?'] == 'HOUSE_GROUP'].count()
data.drop(data[data['House or appartment?'] == 'HOUSE_GROUP'].index, inplace=True)


In [29]:
data['House or appartment?'].unique()

array(['HOUSE', 'APARTMENT'], dtype=object)

3. Check subtype

In [ ]:
data[data['subtype'] == 'APARTMENT_GROUP'].count()
data.drop(data[data['subtype'] == 'APARTMENT_GROUP'].index, inplace=True)
data[data['subtype'] == 'HOUSE_GROUP'].count()
data.drop(data[data['subtype'] == 'HOUSE_GROUP'].index, inplace=True)


In [50]:
data['Number of rooms'].unique()
data['Number of rooms'].describe()?

SyntaxError: invalid syntax (3198066364.py, line 2)